In [ ]:
# compute conservation across each genome for every position on the reference genome

In [3]:
import os
import re
import sys
import csv
import time
import random
import requests
import subprocess
from pathlib import Path

from Bio import SeqIO

In [4]:
base_dir = Path.cwd().parent
data_dir = base_dir / 'data'

reference_fasta_fname = data_dir / 'reference' / 'K03455.1.fasta'
reference_gtf_fname = data_dir / 'reference' / 'K03455.1.gtf'

sequence_dir = data_dir / 'sequences'
assert sequence_dir.exists(), f"sequence_dir does not exist: {sequence_dir}"

annotation_dir = data_dir / 'annotation'
assert annotation_dir.exists(), f"annotation_dir does not exist: {annotation_dir}"

outdir = data_dir / 'conservation'
outdir.mkdir(exist_ok=True)

accession_list_fname = data_dir / 'complete_sequences.2022.accessions'
summary_file =data_dir / "download.summary.tsv"

In [5]:
# software links
soft_dir = base_dir / 'soft'
mafft_bin = "mafft"

In [6]:
# load list of genomes

# read summary tsv line by line using csv module
genomes = {}
with open(summary_file,"r") as inFP:
    reader = csv.reader(inFP, delimiter="\t")
    # skip header
    next(reader)
    for lno, (accid, fasta, gff) in enumerate(reader):
        if accid == "K03455": # skip HXB2
            continue

        if (annotation_dir / accid / f"{accid}.vira.gtf").exists():
            acc_outdir = annotation_dir / accid
            acc_outdir.mkdir(exist_ok=True)
            genome_fasta_fname = sequence_dir / f"{accid}.fasta"
            genome_gtf_fname = acc_outdir / f"{accid}.vira.gtf"
            
            genomes[accid] = (genome_fasta_fname, genome_gtf_fname)

In [7]:
# build a file of all sequence for alignments
qry_fasta_fname = outdir / 'all_sequences.fasta'
with open(qry_fasta_fname, 'w') as outFP:
    for accid, (genome_fasta_fname, genome_gtf_fname) in genomes.items():
        with open(genome_fasta_fname, 'r') as inFP:
            for line in inFP:
                outFP.write(line)

In [ ]:
# run mafft
# mafft --6merpair --addfragments othersequences referencesequence > output
msa_fname = outdir / 'all_sequences.msa'
cmd = [mafft_bin, '--6merpair', '--addfragments', str(qry_fasta_fname), str(reference_fasta_fname)]
print(" ".join(cmd) + " > " + str(msa_fname))
# with open(msa_fname, 'w') as f:
#     subprocess.run(cmd, stdout=f)

mafft --6merpair --addfragments /home/avaraby1/HIV_Atlas/data/conservation/all_sequences.fasta /home/avaraby1/HIV_Atlas/data/reference/K03455.1.fasta > /home/avaraby1/HIV_Atlas/data/conservation/all_sequences.msa


nadd = 3090
ppenalty_ex = -10
nthread = 0
blosum 62 / kimura 200
sueff_global = 0.100000
norg = 1
njobc = 2
generating a scoring matrix for nucleotide (dist=200) ... done


Making a distance matrix ..

There are 39208 ambiguous characters
    1 / 1
done.

fTEP 499 / 3090                    
STEP 500 / 3090                    
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
STEP 600 / 3090                    
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
STEP 700 / 3090                    
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
STEP 800 / 3090                    
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
STEP 900 / 3090                    
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
STEP 1000 / 3090       

In [ ]:
# load a matrix from the msa where columns are positions on the reference genome
# rows are genomes
# and values are tuple of (genome position, nucleotide) at that position

def load_msa_as_matrix(fasta_file):
    # Load the alignment
    sequences = list(SeqIO.parse(fasta_file, "fasta"))
    alignment_length = len(sequences[0].seq)
    for seq_record in sequences:
        if len(seq_record.seq) != alignment_length:
            raise ValueError("All sequences in the alignment must be the same length.")

    # Create a matrix where each row corresponds to a genome
    matrix = []
    idx = {} # map of accession to row in matrix
    for seq_index, seq_record in enumerate(sequences):
        idx[seq_record.id] = seq_index
        row = []
        genome_position = 0  # Tracks the position in the original genome
        for align_position, nucleotide in enumerate(seq_record.seq):
            if nucleotide != "-":  # Skipping gaps
                genome_position += 1
            row.append((genome_position, nucleotide))
        matrix.append(row)

    return matrix, idx

msa_mat, msa_idx = load_msa_as_matrix(msa_fname)


In [17]:
# now cleanup the matrix by removing all column which do not contain reference genome nucleotide

def cleanup_matrix(mat, ref_idx):
    assert len(mat) > 0, "Matrix must have at least one row."
    assert ref_idx > 0 or ref_idx < len(mat), "Reference index is out of bounds."

    reference_row = mat[ref_idx]
    columns_to_keep = [i for i in range(len(reference_row)) if reference_row[i][1] != '-']

    clean_mat = []
    for row in mat:
        clean_row = [row[i] for i in columns_to_keep]
        clean_mat.append(clean_row)

    return clean_mat


ref_msa_idx = msa_idx["K03455.1"]
clean_msa_mat = cleanup_matrix(msa_mat, ref_msa_idx)



In [20]:
# compute simple conservation: for each position - count the number of genoems where the nucleotide is the same as the reference genome
def compute_simple_conservation(mat, ref_idx):
    assert len(mat) > 0, "Matrix must have at least one row."
    assert ref_idx > 0 or ref_idx < len(mat), "Reference index is out of bounds."
    ref_row = mat[ref_idx]
    
    conservation_counts = []
    
    # Iterate over each column (position)
    for col_index in range(len(ref_row)):
        ref_nuc = ref_row[col_index][1]
        count = 0
        
        for row in mat:
            if row[col_index][1] == ref_nuc:
                count += 1
                
        conservation_counts.append(count)
    
    return conservation_counts

simple_conservation = compute_simple_conservation(clean_msa_mat, ref_msa_idx)


In [24]:
# write result as bed format
bed_fname = outdir / 'K03455.1.conservation.simple.all.bed'
with open(bed_fname, 'w') as outFP:
    for i, count in enumerate(simple_conservation):
        outFP.write(f"K03455.1\t{i}\t{i+1}\t-\t{count}\t+\n")

In [ ]:
# now we shall also consider the donor/acceptor information
# specifically, for each donor and acceptor site on the reference genome
# we should find out the number of genomes where the corresponding position also has a donor/acceptor site

# load all genome annotations